In [10]:
from anchor import utils
from anchor import anchor_tabular
import pandas as pd
import sklearn.ensemble
import numpy as np
from anchor import anchor_tabular
import os

In [11]:
train_path = "../../datasets/bank_loan/credit_train.csv"
train_data = pd.read_csv(train_path)
#print(train_data.nunique(axis=0))

#print(train_data.isnull().sum())
#print(train_data.shape)
train_data["Months since last delinquent"] = train_data["Months since last delinquent"].fillna('999')
train_data.dropna(inplace=True)
#print(train_data.isnull().sum())
#print(train_data.shape)

In [12]:
# if dataset_name == "bank_loan":
feature_names = ["Loan ID", "Customer ID", "Loan Status", "Current Loan Amount", 
                  "Term", "Credit Score", "Annual Income", "Years in current job", 
                  "Home Ownership","Purpose","Monthly Debt","Years of Credit History",
                  "Months since last delinquent","Number of Open Accounts", 
                  "Number of Credit Problems", "Current Credit Balance", 
                  "Maximum Open Credit", "Bankruptcies", "Tax Liens"]
#feature_names = None
features_to_use = [3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18]
categorical_features = [4, 7, 8, 9]

target_idx = 2
dataset_folder = train_path = "../../datasets/bank_loan/"
discretize = True
balance = True

purpose_map = {
    "Debt Consolidation" : "Debt Consolidation", 
    "other" : "other", 
    "Home Improvements" : "Consumer credit", 
    "Other": "other", 
    "Business Loan": "Business",
    "Buy a Car": "Consumer credit", 
    "Medical Bills":"Medical", 
    "Buy House": "House", 
    "Take a Trip": "Consumer credit",
    "major_purchase": "Consumer credit",
    "small_business": "Business",
    "moving": "Consumer credit", 
    "wedding": "Consumer credit",
    "vacation": "Consumer credit",
    "Educational Expenses": "Consumer credit",
    "renewable_energy": "Consumer credit"
}

transformations = {
  9 : lambda x: utils.map_array_values(x, purpose_map)
}

def filter_fn(data):
  df = pd.DataFrame(data)
  # key("Months since last delinquent") = 12
  #df.replace(r'^\s*$', np.NaN, regex=True)
  df = df.replace(b'', np.NaN)
  df = df.replace(b'NA', np.NaN)
  df[12] = df[12].fillna('999')
  df.dropna(inplace=True)
  ###
  df.drop(df[df[2] == b'nan'].index, inplace=True)
  print(df[2].unique())
  return df.values.astype('|S128')

dataset = utils.load_csv_dataset(
            os.path.join(dataset_folder, 'credit_train.csv'), target_idx, ',',
            feature_names=feature_names, features_to_use=features_to_use,
            categorical_features=categorical_features, discretize=discretize,
            balance=balance, feature_transformations=transformations, filter_fn = filter_fn, skip_first=True)

[b'Fully Paid' b'Charged Off']


In [13]:
print(dataset.train[0])

[ 0.  1.  2.  0. 10.  3. 15.  0.  0.  3.  0.  0.  0.  0.  0.]


In [14]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
c.fit(dataset.train, dataset.labels_train)
print(np.unique(dataset.labels_train))
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, c.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, c.predict(dataset.test)))

[0 1]
Train 0.9844210679576312
Test 0.7813682678311499


In [15]:
transformed_data = []

for r in dataset.train:
    transformed_data.append([dataset.categorical_names[i][int(r_)] for i, r_ in enumerate(r)])        

print(transformed_data[0])

train_data = pd.DataFrame(transformed_data, columns=dataset.feature_names)
print(train_data.shape)
nb_vals = 1000

print(dataset.train[0])
print(dataset.train[0].reshape(1, -1))
y=[]
for k in range(0, nb_vals):
    y.append(c.predict(dataset.train[k].reshape(1, -1))[0])


['Current Loan Amount <= 186648.00', 'Short Term', '724.00 < Credit Score <= 741.00', 'Annual Income <= 849110.00', '< 1 year', 'Rent', 'wedding', 'Monthly Debt <= 10260.19', 'Years of Credit History <= 13.50', 'Number of Open Accounts > 14.00', 'Number of Credit Problems <= 0.00', 'Current Credit Balance <= 112746.00', 'Maximum Open Credit <= 271986.00', 'Bankruptcies <= 0.00', 'Tax Liens <= 0.00']
(27473, 15)
[ 0.  1.  2.  0. 10.  3. 15.  0.  0.  3.  0.  0.  0.  0.  0.]
[[ 0.  1.  2.  0. 10.  3. 15.  0.  0.  3.  0.  0.  0.  0.  0.]]


In [16]:
import sys
sys.path.append(sys.path[0] + '/..')

import arg_explainer as ae
from importlib import reload
reload(ae)
import arg_explainer as ae

print(dataset.feature_names)
explainer = ae.ArgTabularExplainer(c, train_data.iloc[0:nb_vals], y, 'bankloan_' + str(nb_vals), compute=False, output_path='../saves')

['Current Loan Amount', 'Term', 'Credit Score', 'Annual Income', 'Years in current job', 'Home Ownership', 'Purpose', 'Monthly Debt', 'Years of Credit History', 'Number of Open Accounts', 'Number of Credit Problems', 'Current Credit Balance', 'Maximum Open Credit', 'Bankruptcies', 'Tax Liens']


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
G = explainer.build_attack_graph(compute=True, display_graph=False)
print('edges per node:', np.mean([len(G.edges(n)) for n in G.nodes()]))

print(sum([len(G.edges(n)) for n in G.nodes()]))
#explainer.export_graph('tgf', '../saves')
explainer.export_graph('asp', '../saves')


6018 5689  args total
bankloan_200
edges per node: 4768.123857521141


In [9]:
explainer.af_analysis()

['bankloan_100_atk_graph.df', 'bankloan_200_atk_graph.df', 'bankloan_75_atk_graph.df'] ['bankloan_100_R_atk.df', 'bankloan_200_R_atk.df', 'bankloan_500_R_atk.df', 'bankloan_75_R_atk.df']
[4822396, 27910213, 222239181, 2129211]


In [23]:
#ext_gen = explainer.extension_generator_from_graph()
ext_gen = explainer.extension_generator_from_sat(file='../saves/bankloan_100_sat.txt')
max_covi_ext = explainer.make_selection('max_covi', ext_gen)

Using pre-processed file...
Reading ../saves/bankloan_100_sat_preprocessed.txt
Time for selection:  231.74285697937012
Len max_cov_exts:  8


In [24]:
#sel = explainer.make_selection(max_covi_ext, 'max_covc')
res = explainer.apply_inference(max_covi_ext, 'universal')
#print(res)

explainer.display_explanations()

id: 0 coverage: 0 Arg 1/0: None
id: 1 coverage: 0 Arg 1/0: None
id: 2 coverage: 0 Arg 1/0: None
id: 3 coverage: 0 Arg 1/0: None
id: 4 coverage: 4 Arg 1/10: frozenset({'Years in current job_10+ years', 'Home Ownership_Rent', 'Current Loan Amount_Current Loan Amount > 549780.00'})
id: 5 coverage: 6 Arg 1/1: frozenset({'Current Loan Amount_Current Loan Amount <= 186648.00', 'Years of Credit History_13.50 < Years of Credit History <= 16.90'})
id: 6 coverage: 0 Arg 1/0: None
id: 7 coverage: 1 Arg 1/1: frozenset({'Current Loan Amount_186648.00 < Current Loan Amount <= 324764.00', 'Years of Credit History_Years of Credit History > 21.70', 'Monthly Debt_16264.19 < Monthly Debt <= 24059.89', 'Annual Income_849110.00 < Annual Income <= 1174371.00'})
id: 8 coverage: 0 Arg 1/0: None
id: 9 coverage: 8 Arg 1/2: frozenset({'Current Credit Balance_112746.00 < Current Credit Balance <= 210444.00', 'Current Loan Amount_Current Loan Amount > 549780.00'})
id: 10 coverage: 0 Arg 1/0: None
id: 11 coverage: 

In [25]:
res_one = explainer.apply_inference(max_covi_ext, 'one')
print(len(res_one))
explainer.display_explanations()

89
id: 0 coverage: 0 Arg 1/0: None
id: 1 coverage: 0 Arg 1/0: None
id: 2 coverage: 0 Arg 1/0: None
id: 3 coverage: 2 Arg 1/1: frozenset({'Number of Open Accounts_Number of Open Accounts > 14.00', 'Current Loan Amount_324764.00 < Current Loan Amount <= 549780.00', 'Years of Credit History_Years of Credit History <= 13.50'})
id: 4 coverage: 4 Arg 1/10: frozenset({'Years in current job_10+ years', 'Home Ownership_Rent', 'Current Loan Amount_Current Loan Amount > 549780.00'})
id: 5 coverage: 6 Arg 1/1: frozenset({'Current Loan Amount_Current Loan Amount <= 186648.00', 'Years of Credit History_13.50 < Years of Credit History <= 16.90'})
id: 6 coverage: 0 Arg 1/0: None
id: 7 coverage: 1 Arg 1/1: frozenset({'Current Loan Amount_186648.00 < Current Loan Amount <= 324764.00', 'Years of Credit History_Years of Credit History > 21.70', 'Monthly Debt_16264.19 < Monthly Debt <= 24059.89', 'Annual Income_849110.00 < Annual Income <= 1174371.00'})
id: 8 coverage: 0 Arg 1/0: None
id: 9 coverage: 8 Arg

## Brouillon

In [ ]:
"""
stream = explainer.build_naive_extensions(method='solver', file='asp_test.txt')
for l in stream:
    print(l)

exts = []
c = 0
with open('asp_test_preprocessed.txt', 'r') as f:
    for l in f.readlines():
        c += 1
        #e_ = np.array(l.split(' ')).astype(int)
        #exts.append([nd[e] for e in e_])
        
print(c)
"""

In [45]:
#Benchmark
import time
import io

t0 = time.time()
with open('asp_test_preprocessed.txt', 'r') as f:
#f = io.open('asp_test_preprocessed.txt', mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True)
    max_cov = 0
    max_cov_exts = []
    c = 0
    for l in f.readlines():
    #stream = explainer.build_naive_extensions(method='solver', file='asp_test.txt')
    #for l in stream:
        c += 1
        try:
            e_ = np.array(l.split(' ')).astype(int)
            #e_ = l
            ext = [arg_map[e] for e in e_]
            ####
            card_cov = sum([len(explainer.covi_by_arg[arg]) for arg in ext])
            if card_cov > max_cov:
                max_cov = card_cov
                max_cov_exts = []
            if card_cov >= max_cov:
                max_cov_exts.append(ext)
        except:
            print("Skipped line no", c, ": ", l)

print("time len method =", time.time()-t0)
print(max_cov, max_cov_exts)

"""
t0 = time.time()
with open('asp_test_preprocessed.txt', 'r') as f:
    covi_by_extension = dict()
    covi = set()
    max_cov = set()
    max_cov_exts = None
    c = 0
    for l in f.readlines():
        c += 1
        try:
            e_ = np.array(l.split(' ')).astype(int)
            ext = [arg_map[e] for e in e_]
            ####
            covi = set.union(*[explainer.covi_by_arg[arg] for arg in ext])
            covi_by_extension.update({frozenset(ext): covi})
            if len(covi) > len(max_cov):
                max_cov = covi
                max_cov_exts = [ext]
            elif len(covi) == len(max_cov):
                max_cov_exts.append(ext)
        except:
            print("Skipped line no", c, ": ", l)

print("time len method =", time.time()-t0)
print(max_cov, max_cov_exts)
"""

Skipped line no 7448908 :  SATISFIABLE

Skipped line no 7448909 :  

Skipped line no 7448910 :  Models       : 7448907

Skipped line no 7448911 :  Calls        : 1

Skipped line no 7448912 :  Time         : 107.700s (Solving: 107.23s 1st Model: 0.00s Unsat: 0.00s

Skipped line no 7448913 :  CPU Time     : 107.630s

time len method = 384.88955426216125
484 [[frozenset({37, 22, 31}), frozenset({24, 36, 5, 22}), frozenset({24, 34, 37, 22}), frozenset({1, 11, 22}), frozenset({24, 5, 36, 37}), frozenset({35, 4, 31}), frozenset({34, 43, 4}), frozenset({34, 4, 44}), frozenset({42, 45, 39}), frozenset({0, 24, 35, 29}), frozenset({1, 20, 29, 36}), frozenset({24, 5, 37, 22}), frozenset({5, 13}), frozenset({1, 20, 29, 9}), frozenset({17, 44}), frozenset({43, 12}), frozenset({11, 22, 30}), frozenset({35, 5, 30}), frozenset({1, 44, 33}), frozenset({2, 28, 45}), frozenset({32, 36, 44}), frozenset({0, 33, 45}), frozenset({2, 20, 30}), frozenset({43, 4, 20}), frozenset({35, 43, 20}), frozenset({34, 30

'\nt0 = time.time()\nwith open(\'asp_test_preprocessed.txt\', \'r\') as f:\n    covi_by_extension = dict()\n    covi = set()\n    max_cov = set()\n    max_cov_exts = None\n    c = 0\n    for l in f.readlines():\n        c += 1\n        try:\n            e_ = np.array(l.split(\' \')).astype(int)\n            ext = [arg_map[e] for e in e_]\n            ####\n            covi = set.union(*[explainer.covi_by_arg[arg] for arg in ext])\n            covi_by_extension.update({frozenset(ext): covi})\n            if len(covi) > len(max_cov):\n                max_cov = covi\n                max_cov_exts = [ext]\n            elif len(covi) == len(max_cov):\n                max_cov_exts.append(ext)\n        except:\n            print("Skipped line no", c, ": ", l)\n\nprint("time len method =", time.time()-t0)\nprint(max_cov, max_cov_exts)\n'

In [9]:
stream = explainer.build_naive_extensions(method='solver', file='asp_test.txt')
print(stream)
for l in stream:
    print(l)

<generator object ArgTabularExplainer.build_naive_extensions at 0x000001B1092AA3C0>
Pre-processing file...
Done
Streaming  asp_test_preprocessed.txt


In [ ]:
explainer.set_strategy('max_covc', 'existence')
## Define strategy before calling explainer.explain()
explainer.display_explanations()

0.4019384124647283
Number of extensions: 1326480
Time spent on naive extensions: 55.99906277656555 s ( 86.23851943016052 )
len(max_cov_exts)= 1326478 / 1326480
Covc strategy's coverage: 2
Top 5 covs: [2, 2, 2, 2, 2]
id: 0 coverage: 1 Arg 1/12: frozenset({'Relationship_Husband', 'Country_United-States', 'Education_Some-college'})
id: 1 coverage: 1 Arg 1/10: frozenset({'Marital Status_Never-married', 'Age_37.00 < Age <= 48.00'})
id: 2 coverage: 1 Arg 1/22: frozenset({'Age_Age > 48.00', 'Occupation_Exec-managerial', 'Capital Gain_0', 'Workclass_Private'})
id: 3 coverage: 1 Arg 1/18: frozenset({'Hours per week_Hours per week <= 40.00', 'Occupation_Machine-op-inspct'})
id: 4 coverage: 1 Arg 1/17: frozenset({'Capital Loss_0', 'Capital Gain_0', 'Occupation_Sales', 'Hours per week_Hours per week <= 40.00', 'Relationship_Husband'})
id: 5 coverage: 1 Arg 1/8: frozenset({'Occupation_Farming-fishing', 'Hours per week_Hours per week <= 40.00'})
id: 6 coverage: 1 Arg 1/24: frozenset({'Education_Bach

In [ ]:
explainer.set_strategy('max_covc', 'universal')
## Define strategy before calling explainer.explain()
explainer.display_explanations()

0.40616693248272195
Number of extensions: 521076
len(max_cov_exts)= 521074 / 521076
Covc strategy's coverage: 2
Top 5 covs: [2, 2, 2, 2, 2]
id: 0 coverage: 0 Arg 1/0: None
id: 1 coverage: 0 Arg 1/0: None
id: 2 coverage: 0 Arg 1/0: None
id: 3 coverage: 0 Arg 1/0: None
id: 4 coverage: 0 Arg 1/0: None
id: 5 coverage: 0 Arg 1/0: None
id: 6 coverage: 0 Arg 1/0: None
id: 7 coverage: 0 Arg 1/0: None
id: 8 coverage: 0 Arg 1/0: None
id: 9 coverage: 0 Arg 1/0: None
id: 10 coverage: 0 Arg 1/0: None
id: 11 coverage: 0 Arg 1/0: None
id: 12 coverage: 0 Arg 1/0: None
id: 13 coverage: 0 Arg 1/0: None
id: 14 coverage: 0 Arg 1/0: None
id: 15 coverage: 0 Arg 1/0: None
id: 16 coverage: 0 Arg 1/0: None
id: 17 coverage: 0 Arg 1/0: None
id: 18 coverage: 0 Arg 1/0: None
id: 19 coverage: 0 Arg 1/0: None
id: 20 coverage: 0 Arg 1/0: None
id: 21 coverage: 0 Arg 1/0: None
id: 22 coverage: 0 Arg 1/0: None
id: 23 coverage: 0 Arg 1/0: None
id: 24 coverage: 0 Arg 1/0: None
id: 25 coverage: 0 Arg 1/0: None
id: 26 cover

In [ ]:
explainer.set_strategy('max_covi', 'universal')
## Define strategy before calling explainer.explain()
explainer.display_explanations()

0.40616693248272195
Number of extensions: 521076
len(max_cov_exts)= 12 / 521076
Covi strategy's coverage: 60
Top 5 covs: [58, 58, 58, 58, 58]
id: 0 coverage: 0 Arg 1/0: None
id: 1 coverage: 5 Arg 1/4: frozenset({'Relationship_Not-in-family', 'Occupation_Exec-managerial'})
id: 2 coverage: 0 Arg 1/0: None
id: 3 coverage: 3 Arg 1/1: frozenset({'Occupation_Machine-op-inspct', 'Age_28.00 < Age <= 37.00'})
id: 4 coverage: 0 Arg 1/0: None
id: 5 coverage: 2 Arg 1/1: frozenset({'Occupation_Farming-fishing', 'Age_28.00 < Age <= 37.00'})
id: 6 coverage: 2 Arg 1/1: frozenset({'Sex_Female', 'Occupation_Protective-serv'})
id: 7 coverage: 3 Arg 1/1: frozenset({'Occupation_Sales', 'Education_Bachelors'})
id: 8 coverage: 2 Arg 1/1: frozenset({'Occupation_?', 'Sex_Male'})
id: 9 coverage: 3 Arg 1/2: frozenset({'Occupation_Prof-specialty', 'Education_Bachelors', 'Workclass_Private'})
id: 10 coverage: 4 Arg 1/1: frozenset({'Occupation_Protective-serv', 'Sex_Male'})
id: 11 coverage: 1 Arg 1/1: frozenset({'A

In [ ]:
anchor_explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)
idx = 0
np.random.seed(1)
instance = dataset.train[idx].reshape(1, -1)[0]
print("Anchor")
print([dataset.categorical_names[i][int(r_)] for i, r_ in enumerate(instance)])

print('Prediction: ', anchor_explainer.class_names[c.predict(dataset.train[0].reshape(1, -1))[0]])
exp = anchor_explainer.explain_instance(dataset.train[idx], c.predict, threshold=0.95)
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

print("Arg:", idx)
print(explainer.explain_instance(idx))

Anchor
['Age > 48.00', 'Self-emp-not-inc', 'Some-college', 'Married-civ-spouse', 'Sales', 'Husband', 'White', 'Male', '2', '0', '40.00 < Hours per week <= 45.00', 'United-States']
Prediction:  b'>50K'
Anchor: Capital Gain = 2
Precision: 1.00
Coverage: 0.07
Arg: 0
id: 0 coverage: 10 Args5/5: {frozenset({'Occupation_Sales', 'Age_Age > 48.00'}), frozenset({'Workclass_Self-emp-not-inc', 'Age_Age > 48.00'}), frozenset({'Workclass_Self-emp-not-inc', 'Occupation_Sales'}), frozenset({'Education_Some-college', 'Workclass_Self-emp-not-inc'}), frozenset({'Education_Some-college', 'Occupation_Sales', 'Country_United-States'})}
None
